In [2]:
### Nearshore Coastal Condition for Base BP 2022 - 
##### there is some pre-work in excel.  Pull the CCA data from the website, extract
##### the EFTI, SQI WQI and M_AMBI.  Use both 2010 and 2015 data to increase spatial coverage.  
##### Join with EPA site locations and import to ArcPro.   

In [4]:
#### define some inputs
sr= arcpy.SpatialReference(5070)
OutWorkspace = r"E:\projects\BP\Indicators\indicators.gdb"
SEextent= r"E:\projects\BP\1_Extent\BaseBlueprintExtent2022.tif"

In [ ]:
##### Get bathy data from National Coastal Relief Model.  https://www.ngdc.noaa.gov/mgg/coastal/crm.html
##### download vols 2,3,4,5 as net cdf
##### import using "make NetCDF Raster Layer" in ArcPro

In [ ]:
arcpy.md.MakeNetCDFRasterLayer(r"E:\generalgis\bathy\crm_vol5.nc", "z", "x", "y", "crm_vol5", '', None, "BY_VALUE", "CENTER")
arcpy.md.MakeNetCDFRasterLayer(r"E:\generalgis\bathy\crm_vol4.nc", "z", "x", "y", "crm_vol4", '', None, "BY_VALUE", "CENTER")
arcpy.md.MakeNetCDFRasterLayer(r"E:\generalgis\bathy\crm_vol3.nc", "z", "x", "y", "crm_vol3", '', None, "BY_VALUE", "CENTER")
arcpy.md.MakeNetCDFRasterLayer(r"E:\generalgis\bathy\crm_vol2.nc", "z", "x", "y", "crm_vol2", '', None, "BY_VALUE", "CENTER")

In [1]:
##### merge into one SEbathy file
with arcpy.EnvManager(extent="DEFAULT"):
    arcpy.management.MosaicToNewRaster("crm_vol2;crm_vol3;crm_vol4;crm_vol5", r"E:\generalgis\bathy", "SE_bathy_v1.tif", sr, "32_BIT_FLOAT", None, 1, "MEAN", "FIRST")

In [ ]:
##### extract 0-10m
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster="BaseBlueprintExtent2022.tif", cellSize=SEextent):
    arcpy.ddd.Reclassify("SE_bathy_v1.tif", "Value", "-4369.200195 -10 NODATA;-10 0 1;0 2655 NODATA", r"E:\generalgis\bathy\SEBP_bathy_0_10m_v1.tif", "NODATA")

In [ ]:
#### create a new NLCD mask - both of these take "a while"
with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
    arcpy.ddd.Reclassify("NLCD 2019 nonwater_v4.tif", "Value", "1 NODATA;2 1;NODATA 1", r"E:\generalgis\NLCD\NLCD_mask.tif", "DATA")
##### mask land
with arcpy.EnvManager(outputCoordinateSystem=sr, mask="NLCD_mask.tif"):
    out_raster = arcpy.sa.ExtractByMask("SEBP_bathy_0_10m_v1.tif", "NLCD_mask.tif"); out_raster.save(r"E:\projects\BP\Indicators\SEBP_bathy_0_10m_LandMask.tif")

In [4]:
#### create polygon file from raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEextent, outputZFlag="Disabled", outputMFlag="Disabled"):
    arcpy.conversion.RasterToPolygon("SEBP_bathy_0_10m_LandMask.tif", r"E:\generalgis\bathy\SEBP_0-10mbathy_v1.shp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

In [2]:
#### select polygons that intersect CCA locations
outpoly1 = arcpy.management.SelectLayerByLocation("SEBP_0-10mbathy_v1", "INTERSECT", "SEBP_CCA20102015_v3ALB", None, "NEW_SELECTION", "NOT_INVERT")

In [3]:
# recode as in or out based on intersect with CCA points
outpoly1 = arcpy.management.CalculateField("SEBP_0-10mbathy_v1", "ccainter", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

In [4]:
#### save vector to new file
arcpy.conversion.FeatureClassToFeatureClass(outpoly1, OutWorkspace, "SEBP_0_10mbathy_CCApoly_out1", '', 'Id "Id" true true false 10 Long 0 10,First,#,SEBP_0-10mbathy_v1,Id,-1,-1;gridcode "gridcode" true true false 10 Long 0 10,First,#,SEBP_0-10mbathy_v1,gridcode,-1,-1', '')

<Result 'E:\\projects\\BP\\Indicators\\indicators.gdb\\SEBP_0_10mbathy_CCApoly_out1'>

In [5]:
## convert vector to new raster "estuary with no data"
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\yallen\Documents\ArcGIS\Projects\CCA_interp\Default.gdb", workspace=r"C:\Users\yallen\Documents\ArcGIS\Projects\CCA_interp\Default.gdb"):
    arcpy.conversion.FeatureToRaster(outpoly1, "ccainter", r"E:\projects\BP\Indicators\estuary_with_no_data.tif", r"E:\projects\BP\Southeast_Blueprint_2022_Data_Download\SEBlueprint20230109\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif")

In [1]:
#### convert polygon to polylines
#### for some reason this step doesn't run in Pro, but runs fine in ArcMap
#with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
#    arcpy.xtagp.PolygonsToPolylinesTool("SEBP_0_10mbathy_CCApoly", r"E:\projects\BP\Indicators\indicators.gdb\SEBP_0_10mbathy_CCApolyline", "NONE", '')
#### I ran in ArcMap and added back to this project

In [3]:
#### Have to run spline with barriers in two chunks - GOM and SA
#### SA
with arcpy.EnvManager(scratchWorkspace=OutWorkspace, outputCoordinateSystem=sr, snapRaster=SEextent, extent="1361055.8194 914384.1468 1845375.8194 2100554.1468", cellSize=SEextent, workspace=r"E:\projects\BP\indicator review\Default.gdb"):
    arcpy.ddd.SplineWithBarriers("SEBP_CCA20102015_v3ALB", "mean_index", "SEBP_0_10mbathy_CCApolyline_SA", 30, r"E:\projects\BP\Indicators\SEBP_CCA_splinewithbarr_SA.tif", 0.5)
#### GOM
with arcpy.EnvManager(scratchWorkspace=OutWorkspace, outputCoordinateSystem=sr, snapRaster=SEextent, extent="-173594.180600001 264824.1468 1607505.8194 927104.1468", cellSize=SEextent, workspace=r"E:\projects\BP\indicator review\Default.gdb"):
    arcpy.ddd.SplineWithBarriers("SEBP_CCA20102015_v3ALB", "mean_index", "SEBP_0_10mbathy_CCApolyline_GOM_v2", 30, r"E:\projects\BP\Indicators\SEBP_CCA_splinewithbarr_GOM_v2.tif", 0.5)

In [6]:
#### mosaic GOM and SA to new raster (SE) 
with arcpy.EnvManager(extent="DEFAULT"):
    arcpy.management.MosaicToNewRaster("SEBP_CCA_splinewithbarr_GOM_v2.tif;SEBP_CCA_splinewithbarr_SA.tif", r"E:\projects\BP\Indicators", "SEBP_CCA_splinewithbarr_SE_v2.tif", sr, "32_BIT_FLOAT", 30, 1, "MEAN", "FIRST")

In [7]:
## mask with "estuary with no data" to get znd values
outraster3 = arcpy.ddd.Times("estuary_with_no_data.tif", "SEBP_CCA_splinewithbarr_SE_v2.tif", r"E:\projects\BP\Indicators\EstuarineCoastalCondition_znd_v2.tif")

In [8]:
#### bin using Amys thresholds: 0-2 = 1; 2-2.4 = 2; 2.4-3.7 = 3; 3.7-4 = 4; 4-5 = 5 and save to final format
outraster = arcpy.ddd.Reclassify("EstuarineCoastalCondition_znd_v2.tif", "Value", "0 0.308741 0;0.308741 2 1;2 2.400000 2;2.400000 3.700000 3;3.700000 4 4;4 9 5", r"E:\projects\BP\Indicators\EstuarineCoastalCondition_znd_v2_reclass.tif", "DATA")

In [4]:
#### extract by raster mask
#    out_raster = arcpy.sa.ExtractByMask("EstuarineCoastalCondition_znd.tif", SEextent); out_raster.save(r"E:\projects\BP\Indicators\EstuarineCoastalConditionznd_masked.tif")

In [10]:
arcpy.management.AddField(outraster, "descript", "text", 25)
arcpy.management.AddField(outraster, "Red", "long", 9)
arcpy.management.AddField(outraster, "Green", "long", 9)
arcpy.management.AddField(outraster, "Blue", "long", 9)

<Result 'E:\\projects\\BP\\Indicators\\EstuarineCoastalCondition_znd_v2_reclass.tif'>

In [11]:
# set code block for next step
codeblock = """
def Reclass(Value):
    if Value == 5:
        return "5 = Good"
    elif Value == 4:
        return "4 = Fair to good"
    elif Value == 3:
        return "3 = Fair"
    elif Value == 2:
        return "2 = Poor to fair"
    elif Value == 1:
        return "1 = Poor"
    elif Value == 0:
        return "0 = Shallow estuary not assessed for condition"    
"""

In [12]:
arcpy.management.CalculateField(outraster, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'E:\\projects\\BP\\Indicators\\EstuarineCoastalCondition_znd_v2_reclass.tif'>

In [13]:
# Value R G B
# 0 255 255 255
# 1 255 232 133
# 2 198 183 93
# 3 91 147 183
# 4 61 113 169
# 5 58 77 107

# set code block for next step
codeblock_red = """
def Reclass(Value):
    if Value == 5:
        return "58"
    elif Value == 4:
        return "61"
    elif Value == 3:
        return "91"
    elif Value == 2:
        return "198"
    elif Value == 1:
        return "255"
    elif Value == 0:
        return "255"
        
"""
codeblock_green = """
def Reclass(Value):
    if Value == 5:
        return "77"
    elif Value == 4:
        return "113"
    elif Value == 3:
        return "147"
    elif Value == 2:
        return "183"
    elif Value == 1:
        return "232"
    elif Value == 0:
        return "255"
"""
codeblock_blue = """
def Reclass(Value):
    if Value == 5:
        return "107"
    elif Value == 4:
        return "169"
    elif Value == 3:
        return "183"
    elif Value == 2:
        return "93"
    elif Value == 1:
        return "133"
    elif Value == 0:
        return "255"
"""

In [14]:
arcpy.management.CalculateField(outraster, "Red", "Reclass(!Value!)", "PYTHON3", codeblock_red, "LONG")
arcpy.management.CalculateField(outraster, "Green", "Reclass(!Value!)", "PYTHON3", codeblock_green, "LONG")
arcpy.management.CalculateField(outraster, "Blue", "Reclass(!Value!)", "PYTHON3", codeblock_blue, "LONG")

<Result 'E:\\projects\\BP\\Indicators\\EstuarineCoastalCondition_znd_v2_reclass.tif'>

In [16]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEextent, snapRaster=SEextent, cellSize=SEextent):
    arcpy.management.CopyRaster(outraster, r"E:\projects\BP\Indicators\EstuarineCoastalCondition_znd_final.tif", '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")